This file uses the run generated by `10_year_scale_run.py` and floweaver
to produce a Sankey diagram that maps appointments with HSI events via facility levels.

Below is the instruction to run the file.

### Install floweaver in Anaconda Prompt (if use Jupyter Notebook) / PyCharm Terminal:

pip install floweaver

pip install ipysankeywidget

jupyter nbextension enable --py --sys-prefix ipysankeywidget

jupyter notebook (to open jupyter notebook, which should be installed first) 

### To display and save the output figures:
Select Start Jupyter Server from the Jupyter Actions Menu (lightbulb icon next to Run All cells icon) -> Open Event Log -> Open in Browser
Or 
Type jupyter notebook in PyCharm Terminal

Find the script and run all cells


In [ ]:
import tlo

from pathlib import Path

from tlo.analysis.utils import get_scenario_outputs, load_pickled_dataframes

import pandas as pd

import numpy as np

from matplotlib import pyplot as plt

from ipysankeywidget import SankeyWidget

from floweaver import *

In [ ]:
# Declare the name of the file that specified the scenarios used in this run.
scenario_filename = '10_year_scale_run.py'

# Declare usual paths:
# Get the tlo path
tlopath = Path(tlo.__file__).parent.parent.parent
outputspath = tlopath / Path('outputs/bshe@ic.ac.uk')

# Find results folder (most recent run generated using that scenario_filename)
f = -4  # -4: Actual + Default health care seeking, -2: Actual + Maximal health care seeking
results_folder = get_scenario_outputs(scenario_filename, outputspath)[f]
print(f"Results folder is: {results_folder}")

# Declare path for output graphs from this script
make_graph_file_name = lambda stub: results_folder / f"{stub}.png"  # noqa: E731

# Extract results
hsi = pd.read_csv(results_folder / 'hsi_count_by_treatment_appt_level.csv')
hsi["Module"] = hsi["TREATMENT_ID"].str.split('_').apply(lambda x: x[0])

# todo: Format data

In [ ]:
# Sankey 0 that map appt to hsi considering only appt footprint for each hsi

# Prepare the data: appt type and number per hsi
appt_and_hsi = appts_by_treatment_id_short.reset_index().copy()
appt_and_hsi.rename(columns={'index': 'TREATMENT_ID'}, inplace=True)
appt_and_hsi = pd.melt(appt_and_hsi, id_vars=['TREATMENT_ID'], value_vars=appt_and_hsi.columns[1:],
                      var_name='Appt_Type')

# Define the flow
appt_and_hsi['source'] = 'Appt_Type'
appt_and_hsi['target'] = 'TREATMENT_ID'

size = dict(width=1000, height=800, margins=dict(left=120, right=520))

# Nodes in alphabetic order
# partition_appt_type = Partition.Simple('Appt_Type', np.unique(appt_and_hsi['Appt_Type']))
# partition_treatment_id = Partition.Simple('TREATMENT_ID', np.unique(appt_and_hsi['TREATMENT_ID']))
# if to keep the order in the dataframe
# partition_appt_type = Partition.Simple('Appt_Type', pd.unique(pd.Series(appt_and_hsi['Appt_Type'])))
# partition_treatment_id = Partition.Simple('TREATMENT_ID', pd.unique(pd.Series(appt_and_hsi['TREATMENT_ID'])))
# if to fix the oder of the nodes in the way we want
partition_appt_type = Partition.Simple('Appt_Type', pd.array([
    'IPAdmission', 'InpatientDays', 'Over5OPD', 'Under5OPD',
    'AntenatalFirst', 'ANCSubsequent', 'CompDelivery', 'NormalDelivery',
    'FamPlan', 'MajorSurg', 'ConWithDCSA',
    'MaleCirc', 'NewAdult', 'VCTNegative', 'VCTPositive']))
partition_treatment_id = Partition.Simple('TREATMENT_ID', pd.array([
    'Malaria_treatment_complicated_child', 'Malaria_IPTp',
    'Diarrhoea_Treatment_Inpatient', 'Depression_Antidepressant_Refill',
    'PostnatalSupervisor_NeonatalWardInpatientCare', 'CareOfWomenDuringPregnancy_FirstAntenatalCareContact',
    'CareOfWomenDuringPregnancy_AntenatalOutpatientManagementOfAnaemia',
    'CareOfWomenDuringPregnancy_PostAbortionCaseManagement', 'Labour_ReceivesSkilledBirthAttendanceDuringLabour',
    'Contraception_FamilyPlanningAppt', 'GenericEmergencyFirstApptAtFacilityLevel1',
    'GenericFirstApptAtFacilityLevel0', 'OesophagealCancer_StartTreatment', 'breastCancer_StartTreatment',
    'Hiv_Circumcision', 'Hiv_Treatment_InitiationOrContinuation', 'Hiv_TestAndRefer']))

nodes = {
    'Appt': ProcessGroup(['Appt_Type'], partition_appt_type),
    'HSI': ProcessGroup(['TREATMENT_ID'], partition_treatment_id),
}

bundles = [
    Bundle('Appt', 'HSI'),
]

ordering = [
    ['Appt'],  # left
    ['HSI'],   # right
    ]

# Set the color for each appt type
palette = {'IPAdmission': 'lightsteelblue', 'InpatientDays': 'skyblue',
           'Over5OPD': 'cornflowerblue', 'Under5OPD': 'steelblue',
           'AntenatalFirst': 'plum', 'ANCSubsequent': 'hotpink',
           'CompDelivery': 'tomato', 'NormalDelivery': 'darksalmon',
           'FamPlan': 'gold', 'MajorSurg': 'orange', 'ConWithDCSA': 'mediumpurple',
           'MaleCirc': 'lightgreen', 'NewAdult': 'mediumseagreen',
           'VCTNegative': 'greenyellow', 'VCTPositive': 'olivedrab',
          }

# Sankey diagram definition (SDD)
sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_appt_type)

# Generate and save Sankey
sankey_appt_and_hsi = weave(sdd, appt_and_hsi, palette=palette, measures='value').to_widget(**size)

sankey_appt_and_hsi.auto_save_png(make_graph_file_name('Sankey_appt_and_hsi'))

In [ ]:
# Sankey 1 that maps appt to hsi considering appt footprint for each hsi and number of each hsi

# Prepare the data: total number of appts per hsi for year 2010-2018
num_appt_by_hsi = appts_by_treatment_id_full.copy()
for event in num_appt_by_hsi.index:
    num_appt_by_hsi.loc[event,:] = appts_by_treatment_id_full.loc[event,:] * num_hsi_by_treatment_id.loc[event, 'Number_of_HSI']
num_appt_by_hsi = num_appt_by_hsi.reset_index().copy()
num_appt_by_hsi.rename(columns={'index': 'TREATMENT_ID'}, inplace=True)
num_appt_by_hsi = pd.melt(num_appt_by_hsi, id_vars=['TREATMENT_ID'], value_vars=num_appt_by_hsi.columns[1:],
                      var_name='Appt_Type')

# Define the flow
num_appt_by_hsi['source'] = 'Appt_Type'
num_appt_by_hsi['target'] = 'TREATMENT_ID'

size = dict(width=1000, height=800, margins=dict(left=120, right=520))

partition_appt_type = Partition.Simple('Appt_Type', np.unique(num_appt_by_hsi['Appt_Type']))

partition_treatment_id = Partition.Simple('TREATMENT_ID', np.unique(num_appt_by_hsi['TREATMENT_ID']))

nodes = {
    'Appt': ProcessGroup(['Appt_Type'], partition_appt_type),
    'HSI': ProcessGroup(['TREATMENT_ID'], partition_treatment_id),
}

bundles = [
    Bundle('Appt', 'HSI'),
]

ordering = [
    ['Appt'],  # left
    ['HSI'],   # right
    ]

# Set the color for each appt type
palette = {'IPAdmission': 'lightsteelblue', 'InpatientDays': 'skyblue',
           'Over5OPD': 'cornflowerblue', 'Under5OPD': 'steelblue',
           'AntenatalFirst': 'plum', 'ANCSubsequent': 'hotpink',
           'CompDelivery': 'tomato', 'NormalDelivery': 'darksalmon',
           'FamPlan': 'gold', 'MajorSurg': 'orange', 'ConWithDCSA': 'mediumpurple',
           'MaleCirc': 'lightgreen', 'NewAdult': 'mediumseagreen',
           'VCTNegative': 'greenyellow', 'VCTPositive': 'olivedrab',
          }

# Sankey diagram definition (SDD)
sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_appt_type)

# Generate and save Sankey
sankey_num_appt_by_hsi = weave(sdd, num_appt_by_hsi, palette=palette, measures='value').to_widget(**size)

sankey_num_appt_by_hsi.auto_save_png(make_graph_file_name('Sankey_num_appt_by_hsi'))